<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_generate_color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf

--2021-09-06 13:06:56--  https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf [following]
--2021-09-06 13:06:56--  https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92920 (91K) [application/octet-stream]
Saving to: ‘Dosis-Bold.ttf’

Dosis-Bold.ttf      100%[===================>]  90.74K  --.-KB/s    in 0.009s  

2021-09-06 13:06:57 (9.47 MB/s) - ‘Dosis-Bold.ttf’ saved [92920/92920]



In [53]:
import os
import numpy as np
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import requests
from io import BytesIO
from google.colab import files

colors={'w':(255, 255, 255, 255),
    'k':(0, 0, 0, 255),   #black
    'r':(255, 0, 0, 255),    #red
    '1':(255, 0, 221, 255),  #pink
    'b':(0,0,255,255), #blue
    '2':(0,255,69,255),
    '3':(128,0,128,255), #purple
    'o':(255,165,0,255), #orange
    'y':(255,255,0,255)} #yellow

text="""after i escaped the
dark %okk%and dangerous
tunnel %yk1%in the%3oy% last%wkk% part""" 

root='./subtitles/'
if not os.path.exists(root):
    os.makedirs(root)
imw,imh=1920,250

fnt = ImageFont.truetype("Dosis-Bold.ttf", 115)

In [54]:
text_lines=text.split('\n')
for i,text_line in enumerate(text_lines):
    z = re.match(r"%\w{3}%", text_line)
    if not z:
        text_lines[i]='%wkk%'+text_line
    

In [55]:
i=0
w_start=0

for text_line in text_lines:
    z = re.split(r"(%\w{3}%)", text_line)
    color_strings = z[1::2]
    text_strings = z[2::2]

    full_text_line = ''.join(text_strings)
    w_end = w_start + len(full_text_line.split())
    name = root+f'a_{i:04d}_{w_start:04d}_{w_end:04d}.png'
    i+=1
    w_start = w_end

    im = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d = ImageDraw.Draw(im)
    imface = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d2 = ImageDraw.Draw(imface)




    w,h=d.textsize(full_text_line,fnt)
    x,y = (imw-w)//2, (imh-h)//2
    print(name)
    print(full_text_line)
    prior_text=''
    for color_string, text_string in zip(color_strings,text_strings):
        font_c = color_string[1]
        outline_c = color_string[2]
        shadow_c = color_string[3]

        print(font_c, outline_c, shadow_c, text_string)
        print(d.textsize(prior_text,fnt))

        d.text((x,y), text_string, font=fnt, fill=colors[shadow_c])
        d2.text((x,y), text_string, font=fnt, fill=colors[font_c], stroke_width=7, stroke_fill=colors[outline_c])

        prior_text+=text_string
        x+=d.textsize(text_string,fnt)[0]

    im = im.filter(ImageFilter.GaussianBlur(radius=30))
    imnp_raw=np.asarray(im)
    imnp=imnp_raw.copy()
    imnp[:,:,3]=(imnp_raw[:,:,3]*(255.0/imnp_raw[:,:,3].max())).astype(np.uint8)
    imfacenp = np.asarray(imface)
    blank = np.zeros((imh,imw,4),dtype=np.uint8)
    compnp=np.where((imnp[:,:,3]>0)[:,:,None], imnp, blank)
    compnp=np.where((imfacenp[:,:,3]>200)[:,:,None], imfacenp, compnp)

    Image.fromarray(compnp,'RGBA').save(name)


    print()

./subtitles/a_0000_0000_0004.png
after i escaped the
w k k after i escaped the
(0, 0)

./subtitles/a_0001_0004_0007.png
dark and dangerous
w k k dark 
(0, 0)
o k k and dangerous
(237, 120)

./subtitles/a_0002_0007_0012.png
tunnel in the last part
w k k tunnel 
(0, 0)
y k 1 in the
(327, 120)
3 o y  last
(592, 120)
w k k  part
(785, 120)



In [56]:
%cd /content/subtitles
!zip -r subtitles.zip *
files.download('/content/subtitles/subtitles.zip')


/content/subtitles
  adding: a_0000_0000_0004.png (deflated 5%)
  adding: a_0001_0004_0007.png (deflated 3%)
  adding: a_0002_0007_0012.png (deflated 2%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>